In [1]:
!pip install -U transformers trl datasets torch==2.6.0 accelerate bitsandbytes peft langchain langchain_huggingface pymongo openai pyngrok fastapi uvicorn

In [2]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn
import os
import nest_asyncio
from google.colab import userdata
from rag_chatbot import RAGChatbot

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI(title="HCMIU RAG Chatbot API")

# Initialize RAG chatbot
chatbot = RAGChatbot(
    mongodb_uri=userdata.get('mongodb'),
    openai_api_key=userdata.get('OpenAI')
)

class Query(BaseModel):
    text: str

@app.get("/")
async def read_root():
    return {"message": "HCMIU RAG Chatbot API is running!"}

@app.post("/chat")
async def chat(query: Query):
    try:
        response = chatbot.chat(query.text)
        return {"response": response}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

def start_server():
    # Set up ngrok authtoken
    NGROK_AUTH_TOKEN = userdata.get('ngrok_auth_token')
    if NGROK_AUTH_TOKEN:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        print("ngrok authtoken set.")
    else:
        print("NGROK_AUTH_TOKEN not found in Colab secrets. Please add it to use ngrok.")

    try:
        # Kill any existing ngrok tunnels
        ngrok.kill()
        print("Existing ngrok tunnels killed.")

        # Start a new tunnel
        public_url = ngrok.connect(addr="8000", proto="http")
        print(f"FastAPI Public URL: {public_url}")

        # Start FastAPI server
        uvicorn.run(app, host="0.0.0.0", port=8000, loop="asyncio")
    except Exception as e:
        print(f"Error starting server: {e}")

if __name__ == "__main__":
    start_server()

Loading model and tokenizer...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model and tokenizer loaded successfully!
ngrok authtoken set.
Existing ngrok tunnels killed.
FastAPI Public URL: NgrokTunnel: "https://7218-34-125-166-77.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [9574]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)



Searching for 'Tham gia bài báo khoa học được cộng bao nhiêu điểm rèn luyện ?' in document type: diem_ren_luyen
Found 10 relevant documents

📄 Result 1
Title: Tiêu chí 3.1.4
Document Type: diem_ren_luyen
Combined Score: 0.5885
Vector Score: 1.0000
Text Score: 0.1771
Content Preview: Mô tả: Tham gia sinh hoạt chuyên đề, hội thảo về học thuật, sáng tạo, nghiên cứu khoa học, tăng cường kỹ năng theo hình thức trực tiếp
Điểm: +3
Đơn vị phụ trách: P.CTSV, P.QLKH, Khoa/BM, ĐTN/HSV, ĐVK
--------------------------------------------------

📄 Result 2
Title: Tiêu chí 1.2.1
Document Type: diem_ren_luyen
Combined Score: 0.5156
Vector Score: 0.0312
Text Score: 1.0000
Content Preview: Mô tả: Tham gia 01 hoạt động nghiên cứu khoa học (Hội nghị/Bài báo/Đề tài nghiên cứu khoa học từ cấp Khoa/BM/Phòng/Trung tâm trở lên).
Điểm: +5
Đơn vị phụ trách: P.QLKH, Khoa/BM, ĐVK
--------------------------------------------------

📄 Result 3
Title: Tiêu chí 3.1.5
Document Type: diem_ren_luyen
Combined Score: 0.4409

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [9574]
